# jieba分词

## 分词算法

- 基于统计词典,构造前缀词典，基于前缀词典对句子进行切分，得到所有切分可能，根据切分位置，构造一个有向无环图（DAG）
- 基于DAG图,采用动态规划计算最大概率路径（最有可能的分词结果），根据最大概率路径分词
- 对于新词(词库中没有的词），采用有汉字成词能力的 HMM 模型进行切分。

## jieba分词

In [2]:
import jieba

content = "现如今，机器学习和深度学习带动人工智能飞速的发展，并在图片处理、语音识别领域取得巨大成功。"

### 精确（默认）分词

In [3]:
"/".join(jieba.cut(content, cut_all=False))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.099 seconds.
Prefix dict has been built succesfully.


'现如今/，/机器/学习/和/深度/学习/带动/人工智能/飞速/的/发展/，/并/在/图片/处理/、/语音/识别/领域/取得/巨大成功/。'

### 全模式

把句子中所有的可能是词语的都扫描出来，速度非常快，但不能解决歧义。

In [4]:
"/".join(jieba.cut(content, cut_all=True))

'现如今/如今///机器/学习/和/深度/学习/带动/动人/人工/人工智能/智能/飞速/的/发展///并/在/图片/处理///语音/识别/领域/取得/巨大/巨大成功/大成/成功//'

### 搜索引擎模式

在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词

In [6]:
"/".join(jieba.cut_for_search(content))

'如今/现如今/，/机器/学习/和/深度/学习/带动/人工/智能/人工智能/飞速/的/发展/，/并/在/图片/处理/、/语音/识别/领域/取得/巨大/大成/成功/巨大成功/。'

### 用 lcut 生成 list

In [8]:
print(jieba.lcut(content))

['现如今', '，', '机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '，', '并', '在', '图片', '处理', '、', '语音', '识别', '领域', '取得', '巨大成功', '。']


### 获取词性

In [9]:
import jieba.posseg as psg

[(x.word, x.flag) for x in psg.lcut(content)]

[('现如今', 't'),
 ('，', 'x'),
 ('机器', 'n'),
 ('学习', 'v'),
 ('和', 'c'),
 ('深度', 'ns'),
 ('学习', 'v'),
 ('带动', 'v'),
 ('人工智能', 'n'),
 ('飞速', 'n'),
 ('的', 'uj'),
 ('发展', 'vn'),
 ('，', 'x'),
 ('并', 'c'),
 ('在', 'p'),
 ('图片', 'n'),
 ('处理', 'v'),
 ('、', 'x'),
 ('语音', 'n'),
 ('识别', 'v'),
 ('领域', 'n'),
 ('取得', 'v'),
 ('巨大成功', 'nr'),
 ('。', 'x')]

### 并行分词

#### 开启4个进程

In [13]:
jieba.enable_parallel(4) #不支持Windows

NotImplementedError: jieba: parallel mode only supports posix system

#### 关闭并行模式

In [12]:
jieba.disable_parallel()

### 获取分词表中的top n

In [16]:
from collections import Counter

Counter(jieba.lcut(content)).most_common(5)

[('学习', 2), ('，', 2), ('、', 1), ('。', 1), ('人工智能', 1)]

### 自定义词典和字典

In [17]:
txt = "铁甲网是中国最大的工程机械交易平台。"
jieba.lcut(txt)

['铁甲', '网是', '中国', '最大', '的', '工程机械', '交易平台', '。']

In [18]:
jieba.add_word("铁甲网")
jieba.lcut(txt)

['铁甲网', '是', '中国', '最大', '的', '工程机械', '交易平台', '。']

通过文件添加

In [22]:
jieba.load_userdict("data/user_dict.txt")

## 注意事项

- jieba.cut的第三个参数HMM用来控制是否使用 HMM 模型
- jieba.cut_for_search的第二个参数HMM用来控制是否使用 HMM 模型

# HanLP分词

In [26]:
from pyhanlp import *

print(HanLP.segment(txt))

[铁甲/n, 网/n, 是/vshi, 中国/ns, 最大/gm, 的/ude1, 工程/n, 机械/n, 交易/vn, 平台/n, 。/w]


In [28]:
CustomDictionary.add("铁甲网")
CustomDictionary.add("工程机械", "nz 1024")
CustomDictionary.add("交易平台", "nz 1024 n 1")
print(HanLP.segment(txt))

[铁甲网/nz, 是/vshi, 中国/ns, 最大/gm, 的/ude1, 工程机械/nz, 交易平台/nz, 。/w]
